In [78]:
import os
import albumentations
import numpy as np
import torch.nn as nn
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as pyplot
import os
import argparse
from tqdm import tqdm 
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import utils as vutils
from discriminator import Discriminator
from lpips import LPIPS
from vqgan import VQGAN
from utils import load_data, weights_init

In [79]:
class ImagePaths(Dataset):
    def __init__(self, path, size=None):
        self.size = size

        self.images = [os.path.join(path, file) for file in os.listdir(path)]
        self._length = len(self.images)

        self.rescaler = albumentations.SmallestMaxSize(max_size=self.size)
        self.cropper = albumentations.CenterCrop(height=self.size, width=self.size)
        self.preprocessor = albumentations.Compose([self.rescaler, self.cropper])

    def __len__(self):
        return self._length

    def preprocess_image(self, image_path):
        image = Image.open(image_path)
        if not image.mode == "RGB":
            image = image.convert("RGB")
        image = np.array(image).astype(np.uint8)
        image = self.preprocessor(image=image)["image"]
        image = (image / 127.5 - 1.0).astype(np.float32)
        image = image.transpose(2, 0, 1)
        return image

    def __getitem__(self, i):
        example = self.preprocess_image(self.images[i])
        return example


def load_data(data_path):
    train_data = ImagePaths(data_path, size=256)
    train_loader = DataLoader(train_data, batch_size=3, shuffle=False)
    return train_loader

In [80]:
data_path = r"C:\Users\pedro\OneDrive\Área de Trabalho\vq_gan\flowers\rose"

In [81]:
train_dataset = load_data(data_path)

In [82]:
len(train_dataset)

333

In [83]:
next(iter(train_dataset))

tensor([[[[-0.7725, -0.7725, -0.7725,  ...,  0.1686,  0.1608,  0.1529],
          [-0.7725, -0.7725, -0.7725,  ...,  0.1686,  0.1608,  0.1608],
          [-0.7725, -0.7725, -0.7725,  ...,  0.1686,  0.1686,  0.1686],
          ...,
          [-0.7490, -0.7412, -0.7412,  ..., -0.8039, -0.8118, -0.8118],
          [-0.7490, -0.7490, -0.7412,  ..., -0.8196, -0.8275, -0.8275],
          [-0.7490, -0.7490, -0.7490,  ..., -0.8275, -0.8275, -0.8353]],

         [[-0.7725, -0.7725, -0.7725,  ..., -0.0118, -0.0196, -0.0275],
          [-0.7725, -0.7725, -0.7725,  ..., -0.0118, -0.0196, -0.0196],
          [-0.7725, -0.7725, -0.7725,  ..., -0.0118, -0.0118, -0.0118],
          ...,
          [-0.7569, -0.7490, -0.7490,  ..., -0.9686, -0.9765, -0.9765],
          [-0.7569, -0.7569, -0.7490,  ..., -0.9765, -0.9843, -0.9843],
          [-0.7569, -0.7569, -0.7569,  ..., -0.9843, -0.9843, -0.9922]],

         [[-0.9922, -0.9922, -0.9922,  ..., -0.1529, -0.1608, -0.1686],
          [-0.9922, -0.9922, -

In [86]:
for i in train_dataset:
    print(i.shape)
    break

torch.Size([3, 3, 256, 256])


In [93]:
resolution = 256
attn_resolution = [16]
for i in range(2):
    if resolution in attn_resolution:
        print('oi')
    resolution //= 2

In [94]:
resolution

64